# Precalculation of mean and std for FashionMNIST

In [ ]:
import torch
import torch.nn as nn  
import torch.optim as optim
from torchvision import datasets, transforms

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=False)

mean = 0.0
std = 0.0
num_samples = 0

for images, _ in loader:
    images = images.view(images.size(0), -1) #flattern
    mean += images.mean(dim=1).sum()
    std +=images.std(dim=1).sum()
    num_samples += images.size(0)
    

In [ ]:
import torch
import torch.nn as nn  
import torch.optim as optim
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(), # convert to tensor
    #Normalizing Input x_new = (x-mean)/std 
    transforms.Normalize((0.2860,), (0.3205,)),
    #Data Augmentation
    transforms.RandomHorizontalFlip(p=0.5),
    
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # We split our NN in 2 stages
        # 1. generate features (convolutional transformation)
        # 2. Classify the using FFNN
        # BatchNorm + Postactivation
        
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size= 3, padding=1), # 28*28 -> 28*28
            #nn.BatchNorm2d(32), #preactivation
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2), #28*28 -> 14*14
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2), #14*14 -> 7*7
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),

        )
        self.classifier = nn.Sequential(
            nn.Flatten(), # 2D -> 1D
            nn.Linear(128*7*7, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.feature(x)
        x = self.classifier(x)
        return x

In [ ]:
# Now training 

model = CNN()
lr = 1e-2 # learning rate 
loss = nn.CrossEntropyLoss() # CE because multi-class problem 
optimizer = optim.SGD(model.parameters(), lr = lr)

n_epochs = 3 

for epoch in range(n_epochs):
    model.train() # train mode
    running_loss = 0.0 # loss per epoch 
    for images, labels in train_loader:
        optimizer.zero_grad() # reset the gradients 
        # forward
        outputs = model(images) # calculate outputs
        curr_loss = loss(outputs, labels)
        running_loss += curr_loss
        # backward
        curr_loss.backward() # gradients
        optimizer.step() # update weights and biases
    print(f"Epoch[{epoch + 1}/{n_epochs}], Loss: {running_loss}")

Epoch[1/3], Loss: 394.395751953125
Epoch[2/3], Loss: 249.646484375
Epoch[3/3], Loss: 210.93214416503906
